![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


#### List the top five assignments based on total value of donations, categorized by donor type. The output should include four columns: 1) assignment_name, 2) region, 3) rounded_total_donation_amount rounded to two decimal places, and 4) donor_type, sorted by rounded_total_donation_amount in descending order.

In [49]:
-- highest_donation_assignments

WITH first_grouping AS (
SELECT region,
	assignment_name,
	donor_type,
	ROUND(SUM(amount), 2) AS rounded_total_donation_amount
FROM assignments a
INNER JOIN donations d1
USING(assignment_id)
INNER JOIN donors d2
USING(donor_id)
GROUP BY region, assignment_name, donor_type
ORDER BY assignment_name
)
SELECT 
	assignment_name,
	region,
	rounded_total_donation_amount,
	donor_type
FROM first_grouping
ORDER BY rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


#### Identify the assignment with the highest impact score in each region, ensuring that each listed assignment has received at least one donation. The output should include four columns: 1) assignment_name, 2) region, 3) impact_score, and 4) num_total_donations, sorted by region in ascending order. Include only the highest-scoring assignment per region, avoiding duplicates within the same region.

In [47]:
-- top_regional_impact_assignments
WITH counting AS (
SELECT 
	assignment_name,
	region,
	COUNT(donation_id) AS num_total_donations
FROM assignments a
INNER JOIN donations d
USING(assignment_id)
GROUP BY region, assignment_name
),
ranking AS (
SELECT 
	assignment_name,
	a.region,
	impact_score,
	num_total_donations,
	ROW_NUMBER() OVER(PARTITION BY a.region ORDER BY impact_score DESC) AS don_rank
FROM assignments a
INNER JOIN counting c
USING(assignment_name)
WHERE num_total_donations >= 1
)

SELECT 
	assignment_name,
	region,
	impact_score,
	num_total_donations
FROM ranking
WHERE don_rank = 1
;


,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2
